In [148]:
import re
import hashlib
from string import ascii_uppercase
from pycipher import SimpleSubstitution as SimpleSub
import random
from ngram_score import ngram_score

In [149]:
# from practicalcryptography.com
f = open("quadgrams.txt", "r")
qg = f.read()
fitness = ngram_score(qg) # load our quadgram statistics

In [150]:
# frequency taken from http://en.wikipedia.org/wiki/Letter_frequency
englishLetterFreq = {'e': 12.70, 't': 9.06, 'a': 8.17, 'o': 7.51, 'i': 6.97, 'n': 6.75, 's': 6.33, 'h': 6.09, 'r': 5.99, 'd': 4.25, 'l': 4.03, 'c': 2.78, 'u': 2.76, 'm': 2.41, 'w': 2.36, 'f': 2.23, 'g': 2.02, 'y': 1.97, 'p': 1.93, 'b': 1.29, 'v': 0.98, 'k': 0.77, 'j': 0.15, 'x': 0.15, 'q': 0.10, 'z': 0.07}
ETAOIN  = 'etaoinshrdlcumwfgypbvkjxqz'
LETTERS = 'abcdefghijklmnopqrstuvwxyz'
cipher = ''
corpus = ''
h = ''
hc = ''
no_of_iterations = 10
maxkey=''

In [151]:
def slicer(text):
    return text[:]

def letter_count(text):

     # Returns a dictionary with keys of single letters and values of the count of how many times they appear in the text parameter.
     letterCount = {'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0, 'i': 0, 'j': 0, 'k': 0, 'l': 0, 'm': 0, 'n': 0, 'o': 0, 'p': 0, 'q': 0, 'r': 0, 's': 0, 't': 0, 'u': 0, 'v': 0, 'w': 0, 'x': 0, 'y': 0, 'z': 0}
     
     for letter in text.lower():
         if letter in LETTERS:
             letterCount[letter] = letterCount[letter] + 1
     return letterCount

def get_item_at_index_zero(item):
    return item[0]

def get_freq_order(message):

     # Returns a string of the alphabet letters arranged in order of most frequently occurring in the message parameter.

     # first, get a dictionary of each letter and its frequency count

     letter2freq = letter_count(message)
     # second, make a dictionary of each frequency count to each letter(s) with that frequency

     freq2letter = {}
     for letter in LETTERS:
            if letter2freq[letter] not in freq2letter:
                freq2letter[letter2freq[letter]] = [letter]
            else:
                freq2letter[letter2freq[letter]].append(letter)
     # third, put each list of letters in reverse "ETAOIN" order, and then convert it to a string

     for freq in freq2letter:
         freq2letter[freq].sort(key=ETAOIN.find, reverse=True)
         freq2letter[freq] = ''.join(freq2letter[freq])

     # fourth, convert the freq2letter dictionary to a list of tuple
     # pairs (key, value), then sort them

     freqPairs = list(freq2letter.items())
     freqPairs.sort(key=get_item_at_index_zero, reverse=True)
     

     # fifth, now that the letters are ordered by frequency, extract all the letters for the final string

     freqOrder = []

     for freqPair in freqPairs:
         freqOrder.append(freqPair[1])
     return ''.join(freqOrder)

In [152]:
def manual_swap(cipher):
    key_swapped = 'rafiqgtdsxjomnhpkulezywbvc'
    res = SimpleSub(key_swapped).decipher(cipher)
    return res

In [153]:
with open('ciphertext.txt', 'r') as c:
    cipher = c.read()
    #cipher = "".join(re.findall("[a-zA-Z]+", cipher)).lower()
    cipher = slicer(cipher)

with open('corpus.txt', 'r') as c:
    corpus = c.read()
    #corpus = "".join(re.findall("[a-zA-Z]+", corpus)).lower()
    corpus = slicer(corpus)

with open ('sha256sum.txt') as hf:
    h=hf.read()

#with open ('Hash-FirstThousandAlphabets.txt') as hf:
    #hc=hf.read()
    #print('hc= ', hc)

In [ ]:
sorted_freq_corpus = get_freq_order(corpus)
sorted_freq_cipher = get_freq_order(cipher)
print('Freq Corpus', sorted_freq_corpus,'\nFreq Cipher', sorted_freq_cipher)

ptext_cndt = manual_swap(cipher)
digest = hashlib.sha256(ptext_cndt.encode('utf-8')).hexdigest()
print('digest: ',digest)

if digest == h:
    print('Found \nKey: ', sorted_freq_cipher)
else:
    # cipher = re.sub('[^a-zA-Z]','',cipher.lower())
    cipher = cipher.lower()
    maxkey = list('abcdefghijklmnopqrstuvwxyz')
    maxscore = -99e9
    parentscore,parentkey = maxscore,maxkey[:]
    print ("\n..Brute Forcing the cypher..\nYou may have to wait several iterations")
    i = 0
    while (i<no_of_iterations and digest != hc):
        random.shuffle(parentkey)
        deciphered = SimpleSub(parentkey).decipher(cipher, keep_punct=True).lower()
        parentscore = fitness.score(deciphered)
        count = 0
        while count < 1000:
            a = random.randint(0,25)
            b = random.randint(0,25)
            child = parentkey[:]
            # swap two characters in the child
            child[a],child[b] = child[b],child[a]
            deciphered = SimpleSub(child).decipher(cipher, keep_punct=True).lower()
            score = fitness.score(deciphered)
            # if the child was better, replace the parent with it
            if score > parentscore:
                parentscore = score
                parentkey = child[:]
                count = 0
            count = count+1
        # keep track of best score seen so far, keys with best scores are only used for decryption of the ciphertext now, each iteration and try is displayed
        if parentscore>maxscore:
            i = i+1
            maxscore,maxkey = parentscore,parentkey[:]
            print ('\nbest score so far:',maxscore)
            ss = SimpleSub(maxkey)
            plain_t = ss.decipher(cipher, keep_punct=True)
            print ('    best key: '+''.join(maxkey))
            print ('    plaintext: '+plain_t.lower())
            digest = hashlib.sha256(plain_t.encode('utf-8')).hexdigest()
            print("The digest is : " , digest)
            if digest == h:
            print('Found!')

print('\n..\n..\nThe Correct key: '+''.join(maxkey))

Freq Corpus etaonihsrdluwcmfgypbvkxjqz 
Freq Cipher hnejlasgqurtzmfxvodpcyikbw
digest:  20422f95bad54b3693967cca3d6b4c0cd3181272786a6e491d2772d7143e6992

..Brute Forcing the cypher..
You may have to wait several iterations

best score so far: -2299244.1045789574
    best key: epfuhxvsabyrzljdwqgntcmiok
    plaintext: up at all!

the little princess got up, rang for the maid, and hurriedly and merrily
began to devise and carry out a plan of how princess mary should be
dressed. princess marys self-esteem was wounded by the fact that
the arrival of a suitor agitated her, and still more so by both
her companions not having the least conception that it could be
otherwise. to tell them that she felt ashamed for herself and for them
would be to betray her agitation, while to decline their offers to
dress her would prolong their banter and insistence. she flushed, her
beautiful eyes grew dim, red blotches came on her face, and it took
on the unattractive martyrlike expression it so often wore,